# Ontology Queries

In [1]:
from rdflib import Graph

# Load ontology
g = Graph()
g.parse("ontology_instancias.ttl", format="turtle")  # ajusta o nome do ficheiro

print(f"Ontology loaded with {len(g)} triples.")


Ontology loaded with 98 triples.


### Ontology Overview – Basic Statistics

This query provides a high-level overview of the ontology by reporting the number of modeled causes, symptoms, and maintenance procedures, as well as basic statistics on maintenance costs.


In [2]:
query_stats = """
PREFIX : <http://www.semanticweb.org/admin/ontologies/2025/11/untitled-ontology-6/>

SELECT 
     (COUNT(DISTINCT ?cause) AS ?numCauses)
     (COUNT(DISTINCT ?symptom) AS ?numSymptoms)
     (COUNT(DISTINCT ?proc) AS ?numProcedures)
     (AVG(?cost) AS ?avgCost)
     (MIN(?cost) AS ?minCost)
     (MAX(?cost) AS ?maxCost)
WHERE {
     ?cause a :Cause .
     ?symptom a :Symptom .
     ?proc a :Procedure ;
           :hasSparePartsEUR ?cost .
}
"""

result = g.query(query_stats)

for row in result:
    print(f"Number of causes: {row.numCauses}")
    print(f"Number of symptoms: {row.numSymptoms}")
    print(f"Number of procedures: {row.numProcedures}")
    print(f"Average maintenance cost: {float(row.avgCost):.2f} EUR")
    print(f"Minimum maintenance cost: {row.minCost} EUR")
    print(f"Maximum maintenance cost: {row.maxCost} EUR")


Number of causes: 4
Number of symptoms: 3
Number of procedures: 4
Average maintenance cost: 222.50 EUR
Minimum maintenance cost: 50 EUR
Maximum maintenance cost: 600 EUR


### Query 1 – Causes and Associated Symptoms

This query retrieves all failure causes and the corresponding observable symptoms, validating the causal relationships defined in the ontology.


In [3]:
query_causes_symptoms = """
PREFIX : <http://www.semanticweb.org/admin/ontologies/2025/11/untitled-ontology-6/>

SELECT ?cause ?symptom
WHERE {
  ?cause a :Cause ;
         :causesSymptom ?symptom .
}
"""

for row in g.query(query_causes_symptoms):
    print(f"Cause: {row.cause.split('/')[-1]} → Symptom: {row.symptom.split('/')[-1]}")


Cause: BearingWearHigh → Symptom: HighVibration
Cause: CloggedFilter → Symptom: LowCoolantFlow
Cause: FanFault → Symptom: HighSpindleTemp
Cause: LowCoolingEfficiency → Symptom: HighSpindleTemp


### Query 2 – Causes and Affected Components

This query identifies which components are affected by each failure cause, confirming the correct usage of the `affectsComponent` relationship.


In [4]:
query_causes_components = """
PREFIX : <http://www.semanticweb.org/admin/ontologies/2025/11/untitled-ontology-6/>

SELECT ?cause ?component
WHERE {
  ?cause a :Cause ;
         :affectsComponent ?component .
}
"""

for row in g.query(query_causes_components):
    print(f"Cause: {row.cause.split('/')[-1]} → Component: {row.component.split('/')[-1]}")


Cause: BearingWearHigh → Component: Bearing
Cause: CloggedFilter → Component: Filter
Cause: FanFault → Component: Fan
Cause: LowCoolingEfficiency → Component: CoolingSystem


### Query 3 – Causes and Mitigation Procedures

This query retrieves the maintenance procedures that mitigate each failure cause, demonstrating the core objective of the knowledge graph.


In [5]:
query_cause_procedures = """
PREFIX : <http://www.semanticweb.org/admin/ontologies/2025/11/untitled-ontology-6/>

SELECT ?cause ?procedure
WHERE {
  ?procedure a :Procedure ;
             :mitigatesCause ?cause .
}
"""

for row in g.query(query_cause_procedures):
    print(f"Cause: {row.cause.split('/')[-1]} → Procedure: {row.procedure.split('/')[-1]}")


Cause: CloggedFilter → Procedure: CleanFilter
Cause: LowCoolingEfficiency → Procedure: FlushCoolant
Cause: FanFault → Procedure: RepairFan
Cause: BearingWearHigh → Procedure: ReplaceBearing


### Query 4 – Procedure Attributes

This query lists all maintenance procedures along with their effort, risk, and spare-parts cost, ensuring that procedures are fully specified with quantitative attributes.


In [6]:
query_procedure_attributes = """
PREFIX : <http://www.semanticweb.org/admin/ontologies/2025/11/untitled-ontology-6/>

SELECT ?procedure ?hours ?risk ?cost
WHERE {
  ?procedure a :Procedure ;
             :hasEffortHours ?hours ;
             :hasRiskRating ?risk ;
             :hasSparePartsEUR ?cost .
}
"""

for row in g.query(query_procedure_attributes):
    print(
        f"Procedure: {row.procedure.split('/')[-1]} | "
        f"Hours: {row.hours} | Risk: {row.risk} | Cost: {row.cost}€"
    )


Procedure: CleanFilter | Hours: 1.5 | Risk: 1 | Cost: 50€
Procedure: FlushCoolant | Hours: 2.0 | Risk: 2 | Cost: 90€
Procedure: RepairFan | Hours: 2.0 | Risk: 2 | Cost: 150€
Procedure: ReplaceBearing | Hours: 6.0 | Risk: 3 | Cost: 600€


### Query 5 – Machine and Component Structure

This query explores the structural composition of machines and components, showing how complex systems are built from hierarchical components.


In [7]:
query_structure = """
PREFIX : <http://www.semanticweb.org/admin/ontologies/2025/11/untitled-ontology-6/>

SELECT ?parent ?component
WHERE {
  ?parent :hasComponent ?component .
}
"""

for row in g.query(query_structure):
    print(f"{row.parent.split('/')[-1]} has component {row.component.split('/')[-1]}")


CoolingSystem has component Fan
CoolingSystem has component Filter
M-A has component CoolingSystem
M-A has component Spindle
Spindle has component Bearing
